# Accounting human miRNA overhangs

Input file `hsa.gff3` obtained from miRBase: 
<https://www.mirbase.org/ftp/CURRENT/genomes/hsa.gff3>

## Creating individual lists 

First of all, different lists for each feature - pre-miRs and mature miRNAs - and split the last one into 3 different files:
- `mirna_5p` for the 5p mature miRNAs. 
- `mirna_3p` for the 3p mature miRNAs.  
- `mirna_single` for the pre-miRs with a single mature miRNA.

In addition, names in column 9 will be formatted to contain only the name - no ID, no alias.

In [2]:
## Reading file
with open("hsa.gff3", "r") as f:
    records = [line.strip().split() for line in f.readlines() if line[0] != "#"]

## Formating last col to contain only the name
for rec in records:
    rec[-1] = rec[-1].split("=")[3].split(";")[0]

## Nesting lists to create files
pre_mirna = [rec for rec in records if rec[2] == "miRNA_primary_transcript"]
mirna_5p = [rec for rec in records if rec[2] == "miRNA" and rec[-1].split("-")[-1] == "5p"]
mirna_3p = [rec for rec in records if rec[2] == "miRNA" and rec[-1].split("-")[-1] == "3p"]
mirna_single = [rec for rec in records if rec[2] == "miRNA" and rec[-1].split("-")[-1] != "5p" and rec[-1].split("-")[-1] != "3p"]

## Removing transcript number from the name     
for record in pre_mirna:
    name = record[-1].split("-")
    record[-1] = "-".join(name[:3])

print(f" There are:\n- {len(pre_mirna)} miRNA primary transcripts")
print(f"- {len(mirna_5p)} mature miRNAs annotated as 5p")
print(f"- {len(mirna_3p)} mature miRNA annotated as 3p")
print(f"- {len(mirna_single)} mature miRNA annotated to be the only transcript of the pre-miRNA")



 There are:
- 1918 miRNA primary transcripts
- 992 mature miRNAs annotated as 5p
- 992 mature miRNA annotated as 3p
- 899 mature miRNA annotated to be the only transcript of the pre-miRNA


In the [Accounting human miRNA annotation file](https://github.com/zavolanlab/mirflowz/blob/mirna_accounting/explorations/accounting_mirna_annotations.ipynb) there are the same amount of mature miRNA records, 2883, but the distribution among `5p`, `3p` and `single` are different.
There are 27 more entries in both the `5p` and the `3p` therefore 54 less entries counted as single mature miRNAs.

This difference takes place due to some single-mature miRNA being annotated as `5p` or `3p`. 
If subtracting the extra sequences and adding them where they belong, the they sum up to the number of pre-mirs, 1918 

### Solution

Given this situation, the filtering must be done in another way. We know that if a pre-miR has two mature sequences, the features in the `gff3` file will be ordered as:

```
miRNA_primary_transcript_1
miRNA_1
miRNA_1
miRNA_primary_transcript_2
```

On the other hand, if the pre-miRNA has a single mature form, the `gff3` will look:

```
miRNA_primary_transcript_1
miRNA_1
miRNA_primary_trasncript_2
```

Provided that the naming of the sequences is not consistent, whenever there is a pre-miR with two mature sequences, these will be merged into a single one going from end to end.   

Strandness will be taken into account.

Finally, after this proceedure, we should have one mature sequence for each primary one; this is 1918 of each.

In [41]:
## Creating empty lists to store entries
pre_mirna = []
mature_mirna = []

## Setting values for previous and next entries
prev_rec = None
pos_rec = None

## For each record except the last two...
for idx in range(len(records) - 2):

    ## ..if first entry..
    if prev_rec == None:
        prev_rec = records[idx][2]
        pos_rec = records[idx + 2][2]
        record = [records[idx][0],
                  records[idx][2],
                  records[idx][3],
                  records[idx][4],
                  records[idx][6],
                  records[idx][8]]
        pre_mirna.append(record)

    ## ..otherwise..
    else:
        if records[idx][2] == "miRNA_primary_transcript":
            record = [records[idx][0],
                      records[idx][2],
                      records[idx][3],
                      records[idx][4],
                      records[idx][6],
                      records[idx][8]]
            pre_mirna.append(record)
        else:
            if prev_rec == pos_rec:
                record = [records[idx][0],
                          records[idx][2],
                          records[idx][3],
                          records[idx][4],
                          records[idx][6],
                          records[idx][8]]
                mature_mirna.append(record)
            else:
                if pos_rec == "miRNA":
                    if records[idx][3] < records[idx + 1][4]:
                        record = [records[idx][0],
                                  records[idx][2],
                                  records[idx][3],
                                  records[idx + 1][4],
                                  records[idx][6],
                                  records[idx][8]]
                    elif records[idx][3] > records[idx + 1][4]:
                        record = [records[idx][0],
                                  records[idx][2],
                                  records[idx + 1][3],
                                  records[idx][4],
                                  records[idx][6],
                                  records[idx][8]]
                    mature_mirna.append(record)              
        prev_rec = records[idx][2]
        pos_rec = records[idx + 2][2]

## Reading last two entries
if records[-2][2] == "miRNA_primary_transcript":
    pre_record = [records[-2][0],
                  records[-2][2],
                  records[-2][3],
                  records[-2][4],
                  records[-2][6],
                  records[-2][8]]
    record = [records[-1][0],
              records[-1][2],
              records[-1][3],
              records[-1][4],
              records[-1][6],
              records[-1][8]]
    pre_mirna.append(pre_record)
else:
    if records[-2][3] < records[-1][4]:
        record = [records[-2][0],
                  records[-2][2],
                  records[-2][3],
                  records[-1][4],
                  records[-2][6],
                  records[-2][8]]
    elif records[-2][3] > records[-1][4]:
        record = [records[-2][0],
                  records[-2][2],
                  records[-1][3],
                  records[-2][4],
                  records[-2][6],
                  records[-2][8]]
mature_mirna.append(record)                


print(f" There are:\n- {len(pre_mirna)} miRNA primary transcripts")
print(f"- {len(mature_mirna)} mature miRNAs")

 There are:
- 1918 miRNA primary transcripts
- 1918 mature miRNAs


Now that we have one mature sequence per each pre-miRNA, let's check that the mature sequence lays within the pre-miRNA coordinates.

In [42]:
print(f"The pre-miRNA {pre_mirna[808][-1]}:")
print(f"- Has strandness {pre_mirna[808][-2]}")
print(f"- Starts at position {pre_mirna[808][2]}")
print(f"- Ends at position {pre_mirna[808][3]}")
print(f" \nThe mature miRNA {mature_mirna[808][-1]}:")
print(f"- Has strandness {mature_mirna[808][-2]}")
print(f"- Starts at position {mature_mirna[808][2]}")
print(f"- Ends at position {mature_mirna[808][3]}")
print(f"\n---------------------------------\n")
print(f"The pre-miRNA {pre_mirna[13][-1]}:")
print(f"- Has strandness {pre_mirna[13][-2]}")
print(f"- Starts at position {pre_mirna[13][2]}")
print(f"- Ends at position {pre_mirna[13][3]}")
print(f"\nThe mature miRNA {mature_mirna[13][-1]}:")
print(f"- Has strandness {mature_mirna[13][-2]}")
print(f"- Starts at position {mature_mirna[13][2]}")
print(f"- Ends at position {mature_mirna[13][3]}")

The pre-miRNA hsa-mir-4525:
- Has strandness -
- Starts at position 82668233
- Ends at position 82668307
 
The mature miRNA hsa-miR-4525:
- Has strandness -
- Starts at position 82668281
- Ends at position 82668301

---------------------------------

The pre-miRNA hsa-mir-4252:
- Has strandness -
- Starts at position 6429834
- Ends at position 6429896

The mature miRNA hsa-miR-4252:
- Has strandness -
- Starts at position 6429844
- Ends at position 6429862


From how the lists are made, we know that the pre-miRNA at the n-th position, has its mature sequence on the n-th position of the mature miRNA list.

In [43]:
print(f"The primary transcripts {pre_mirna[13][-1]}, {pre_mirna[97][-1]} and {pre_mirna[170][-1]},\nhave as mature miRNAs {mature_mirna[13][-1]}, {mature_mirna[97][-1]} and {mature_mirna[170][-1]} respectively.")

The primary transcripts hsa-mir-4252, hsa-mir-6738 and hsa-mir-603,
have as mature miRNAs hsa-miR-4252, hsa-miR-6738-5p and hsa-miR-603 respectively.


In [44]:
from collections import defaultdict

overhang_5 = defaultdict(lambda: 0)
overhang_3 = defaultdict(lambda: 0)
overhang_seqs = defaultdict(lambda: 0)


for idx in range(len(pre_mirna)):
    if pre_mirna[idx][-2] == '-':
        over_3 = int(mature_mirna[idx][2]) - int(pre_mirna[idx][2])
        over_5 = int(pre_mirna[idx][3]) - int(mature_mirna[idx][3])
    else:
        over_3 = int(pre_mirna[idx][3]) - int(mature_mirna[idx][3])
        over_5 = int(mature_mirna[idx][2]) - int(pre_mirna[idx][2])

    if over_3 > 20:
        over_3 = "20+"
    if over_5 > 20:
        over_5 = "20+"
    overhang_5[over_5] += 1
    overhang_3[over_3] += 1
    overhang_seqs[pre_mirna[idx][-1]] = [over_5, over_3]

Before creating the tables, let's make some manual checkings using the dictionary `overhang_seqs. Some manual overhangs calculated from the `hsa.gff3` file are:

    - hsa-mir-6808 has a 5p overhang of 5 and a 3p overhang of 0.
    - hsa-mir-4695 has a 5p overhang of 4 and a 3p overhang of 2.
    - hsa-mir-34a has a 5p overhang of 21 and a 3p overhang of 25.
    - hsa-mir-5585 has a 5p overhang of 0 and a 3p overhang of 0.
    - hsa-mir-488 has a 5p overhang of 13 and a 3p overhang of 11.
    - hsa-mir-1278 has a 5p overhang of 49 and a 3p overhang of 10.
    - hsa-mir-3124 has a 5p overhang of 6 and a 3p overhang of 4.

Their corresponding overhangs according the algorithm are:

In [45]:
mirs = ["hsa-mir-6806", 
        "hsa-mir-4695", 
        "hsa-mir-34a", 
        "hsa-mir-5585", 
        "hsa-mir-488", 
        "hsa-mir-1278", 
        "hsa-mir-3124"]

for mir in mirs:
    print(f"- {mir} has a 5p overhang of {overhang_seqs[mir][0]} and a 3p overhang of {overhang_seqs[mir][1]}.")

- hsa-mir-6806 has a 5p overhang of 5 and a 3p overhang of 0.
- hsa-mir-4695 has a 5p overhang of 4 and a 3p overhang of 2.
- hsa-mir-34a has a 5p overhang of 20+ and a 3p overhang of 20+.
- hsa-mir-5585 has a 5p overhang of 0 and a 3p overhang of 0.
- hsa-mir-488 has a 5p overhang of 13 and a 3p overhang of 11.
- hsa-mir-1278 has a 5p overhang of 20+ and a 3p overhang of 10.
- hsa-mir-3124 has a 5p overhang of 6 and a 3p overhang of 4.


Finally, the dictionaries storing the counts will be turned into dataframes to manipulate them in R studio.

In [46]:
import pandas as pd

df3 = pd.DataFrame(overhang_3, index=[0])
df3.to_csv("count_3.csv", index = False, sep = "\t")

df5 = pd.DataFrame(overhang_5, index=[0])
df5.to_csv("count_5.csv", index = False, sep = "\t")

## Results

What follows is the script used in R to create the tables:

```R
library(tidyr)
library(dplyr)

## Creating table for the 3p overhangs
count_3 <- read.csv("count_3.csv", header = F, sep = "\t")
count_3 <- as.data.frame(t(count_3))
row.names(count_3) <- NULL
colnames(count_3) <- c("Overhang", "Count")

count_3 <- count_3 %>% 
                mutate("Fraction" = round(as.numeric(Count) /1918, 4)) %>%
                arrange(as.numeric(Overhang))
count_3$Overhang <- as.numeric(count_3$Overhang)
cum3 <- cumsum(count_3$Fraction)
count_3 <- count_3 %>%
  mutate("Comulative Proportion" = cum3)


## Creating table for the 5p overhangs
count_5 <- read.csv("count_5.csv", header = F, sep = "\t")
count_5 <- as.data.frame(t(count_5))
row.names(count_5) <- NULL
colnames(count_5) <- c("Overhang", "Count")

count_5 <- count_5 %>% 
                mutate("Fraction" = round(as.numeric(Count) /1918, 4)) %>%
                arrange(as.numeric(Overhang))
count_5$Overhang <- as.numeric(count_5$Overhang)
cum5 <- cumsum(count_5$Fraction)
count_5 <- count_5 %>%
  mutate("Comulative Proportion" = cum5)


# Merging tables

both <- inner_join(count_5, count_3,by = "Overhang", suffix = c("_5p", "_3p")) %>%
        arrange(as.numeric(Overhang))
  
both$Overhang[22] <- "20+"



write.table(both, "overhang.txt", quote = F, sep = "\t", row.names = F)
```

The final table looks like:

|Overhang (nts) |Count_5p | Proportion_5p| Cumulative_5p|Count_3p | Proportion_3p| Cumulative_3p| 
|:--------------|:--------|-------------:|-------------:|:--------|-------------:|-------------:| 
|0              |118      |        0.0615|        0.0615|250      |        0.1303|        0.1303| 
|1              |23       |        0.0120|        0.0735|43       |        0.0224|        0.1527| 
|2              |30       |        0.0156|        0.0891|53       |        0.0276|        0.1803| 
|3              |33       |        0.0172|        0.1063|46       |        0.0240|        0.2043| 
|4              |45       |        0.0235|        0.1298|50       |        0.0261|        0.2304| 
|5              |208      |        0.1084|        0.2382|34       |        0.0177|        0.2481| 
|6              |34       |        0.0177|        0.2559|45       |        0.0235|        0.2716| 
|7              |40       |        0.0209|        0.2768|56       |        0.0292|        0.3008| 
|8              |53       |        0.0276|        0.3044|52       |        0.0271|        0.3279| 
|9              |145      |        0.0756|        0.3800|72       |        0.0375|        0.3654| 
|10             |141      |        0.0735|        0.4535|169      |        0.0881|        0.4535| 
|11             |37       |        0.0193|        0.4728|85       |        0.0443|        0.4978| 
|12             |51       |        0.0266|        0.4994|76       |        0.0396|        0.5374| 
|13             |54       |        0.0282|        0.5276|55       |        0.0287|        0.5661| 
|14             |79       |        0.0412|        0.5688|42       |        0.0219|        0.5880| 
|15             |114      |        0.0594|        0.6282|84       |        0.0438|        0.6318| 
|16             |14       |        0.0073|        0.6355|15       |        0.0078|        0.6396| 
|17             |14       |        0.0073|        0.6428|24       |        0.0125|        0.6521| 
|18             |12       |        0.0063|        0.6491|15       |        0.0078|        0.6599| 
|19             |18       |        0.0094|        0.6585|19       |        0.0099|        0.6698| 
|20             |30       |        0.0156|        0.6741|16       |        0.0083|        0.6781| 
|20+            |625      |        0.3259|        1.0000|617      |        0.3217|        0.9998|